In [30]:
import os
import glob
import lasio
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [31]:
# === USER INPUT ===
tops_excel = r"F:\Petrophysics Sabah Basin Malaysia\Halliburton Tops for R2 Score Order Wise.xlsx"        # Path to your Excel file
las_folder = r"\\10.13.24.151\halliberton\todays work halli\fiNAL\76 Wells QC done(27-06-2025)"                  # Folder containing LAS files
output_excel = "formation_r2_summary.xlsx" # Output Excel file

In [32]:
df = pd.read_excel(tops_excel)
df.head(10)

,Wellbore Name,Formation Top,MD,TVDSS
0,ALAB-1,Stage IVE,789.4,NaN
1,ALAB-1,Stage IVD,1214.6,NaN
2,ALAB-1,J SAND,1405.1,NaN
3,ALAB-1,K SAND,1505.8,NaN
4,ALAB-1,M SAND,2002.7,NaN
5,ALUM-1,H110,2016,1963
6,ALUM-1,H136,2128.64,2055.46
7,ALUM-1,Base H150,2385,2260
8,ANDALUSIT-1,IVE,NaN,908.8
9,ANDALUSIT-1,IVD,NaN,1756.8


In [39]:
wells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1893420 entries, 0 to 1893419
Data columns (total 25 columns):
 #   Column            Dtype  
---  ------            -----  
 0   DEPT              float64
 1   CALI_UM           float64
 2   DT_ML_UM          float64
 3   DT_UM             float64
 4   GR_UM             float64
 5   NPHI_UM           float64
 6   P_IMP_UM          float64
 7   P_WAVE_FAUST_UM   float64
 8   RHOB_GARDENER_UM  float64
 9   RHOB_ML_UM        float64
 10  RHOB_UM           float64
 11  RT_UM             float64
 12  WellName          object 
 13  DTS_UM            float64
 14  VPVS_UM           float64
 15  P-WAVE_FAUST_UM   float64
 16  P-WAVE_UM         float64
 17  P_WAVE_UM         float64
 18  P-IMP_ML_UM       float64
 19  P-IMP_UM          float64
 20  VPVS_RATIO        float64
 21  DEPTH:1           float64
 22  DEPTH:2           float64
 23  P_IMP_ML_UM       float64
 24  P_WAVE            float64
dtypes: float64(24), object(1)
memory usage: 361.1

In [33]:
las_files = glob.glob(os.path.join(las_folder, "*.las"))

# Empty list to store DataFrames
dfs = []

# Loop through each LAS file
for file in las_files:
    las = lasio.read(file)
    
    # Convert LAS data to pandas DataFrame
    df = las.df()
    
    # Reset the index so depth becomes a regular column
    df = df.reset_index()
    
    # Rename depth column (if needed)
    #df = df.rename(columns={"DEPT": "Depth"})
    
    # Add a column for the source file name
    well_name = os.path.splitext(os.path.basename(file))[0]
    df["WellName"] = well_name
    
    dfs.append(df)

# Combine all DataFrames into one
wells = pd.concat(dfs, ignore_index=True)

print(f"Loaded {len(las_files)} LAS files.")
print(wells.head())

Loaded 109 LAS files.
       DEPT  CALI_UM  DT_ML_UM    DT_UM    GR_UM  NPHI_UM   P_IMP_UM  \
0  929.4895  18.4808  124.0327  115.326  77.2615   0.3168  5601.8633   
1  929.6419  18.4044  124.0327  115.326  77.2615   0.3168  5601.8633   
2  929.7943  18.5446  123.1210  115.326  64.6554   0.3168  5643.3428   
3  929.9467  18.4171  122.9883  115.326  52.6748   0.3218  5582.8081   
4  930.0991  18.2769  118.1172  115.326  50.0522   0.2899  5796.5732   

   P_WAVE_FAUST_UM  RHOB_GARDENER_UM  RHOB_ML_UM  ...  P-WAVE_FAUST_UM  \
0        1979.3074            2.0564      2.2796  ...              NaN   
1        1979.4371            2.0564      2.2796  ...              NaN   
2        1979.5669            2.0564      2.2796  ...              NaN   
3        1979.6967            2.0564      2.2527  ...              NaN   
4        1948.8062            2.0564      2.2463  ...              NaN   

   P-WAVE_UM P_WAVE_UM  P-IMP_ML_UM  P-IMP_UM  VPVS_RATIO  DEPTH:1  DEPTH:2  \
0        NaN       Na

In [34]:
wells.columns

Index(['DEPT', 'CALI_UM', 'DT_ML_UM', 'DT_UM', 'GR_UM', 'NPHI_UM', 'P_IMP_UM',
       'P_WAVE_FAUST_UM', 'RHOB_GARDENER_UM', 'RHOB_ML_UM', 'RHOB_UM', 'RT_UM',
       'WellName', 'DTS_UM', 'VPVS_UM', 'P-WAVE_FAUST_UM', 'P-WAVE_UM',
       'P_WAVE_UM', 'P-IMP_ML_UM', 'P-IMP_UM', 'VPVS_RATIO', 'DEPTH:1',
       'DEPTH:2', 'P_IMP_ML_UM', 'P_WAVE'],
      dtype='object')

In [29]:
wells['source_file']

0            ALAB-1.LAS
1            ALAB-1.LAS
2            ALAB-1.LAS
3            ALAB-1.LAS
4            ALAB-1.LAS
               ...     
1893415    ZOISIT-1.LAS
1893416    ZOISIT-1.LAS
1893417    ZOISIT-1.LAS
1893418    ZOISIT-1.LAS
1893419    ZOISIT-1.LAS
Name: source_file, Length: 1893420, dtype: object

In [40]:
# --- Input Files ---
formation_excel = r"F:\Petrophysics Sabah Basin Malaysia\Halliburton Tops for R2 Score Order Wise.xlsx"     # Excel with [Wellbore Name, Formation Top, MD, TVDSS]
output_file = "Formation_R2_Scores.xlsx"


# --- Assuming wells dataframe already exists ---
# wells has columns like ['DEPT', 'RHOB_UM', 'RHOB_ML_UM', 'DT_UM', 'DT_ML_UM', 'WellName', ...]

# --- Load Formation Tops ---
tops = pd.read_excel(formation_excel)

# --- Clean column names ---
tops.columns = tops.columns.str.strip().str.lower()
wells.columns = wells.columns.str.strip()

# --- Convert depth columns to numeric ---
tops['md'] = pd.to_numeric(tops['md'], errors='coerce')
tops['tvdss'] = pd.to_numeric(tops['tvdss'], errors='coerce')
wells['DEPT'] = pd.to_numeric(wells['DEPT'], errors='coerce')

results = []

# --- Process each well individually ---
for well_name in tops['wellbore name'].unique():
    well_tops = tops[tops['wellbore name'] == well_name].copy()
    well_data = wells[wells['WellName'].str.lower() == well_name.lower()].copy()

    if well_data.empty:
        print(f"⚠️ Skipping {well_name}: No matching LAS data found.")
        continue

    # Use MD if available, otherwise TVDSS
    well_tops['use_depth'] = np.where(well_tops['md'].notna(), well_tops['md'], well_tops['tvdss'])
    well_tops = well_tops.sort_values('use_depth').reset_index(drop=True)

    # Remove invalid or non-numeric depths
    well_tops = well_tops[well_tops['use_depth'].notna()]
    if len(well_tops) < 2:
        print(f"⚠️ Skipping {well_name}: Not enough formation intervals with valid depths.")
        continue

    # --- Formation-wise loop ---
    for i in range(len(well_tops) - 1):
        formation = well_tops.loc[i, 'formation top']
        top_depth = float(well_tops.loc[i, 'use_depth'])
        base_depth = float(well_tops.loc[i + 1, 'use_depth'])

        # Ensure top < base
        if base_depth <= top_depth:
            continue

        # --- Filter data for this formation ---
        formation_data = well_data[(well_data['DEPT'] >= top_depth) & (well_data['DEPT'] < base_depth)]

        if formation_data.empty:
            continue

        # --- Compute RHOB R² ---
        if {'RHOB_UM', 'RHOB_ML_UM'}.issubset(formation_data.columns):
            rhob = formation_data[['RHOB_UM', 'RHOB_ML_UM']].dropna()
            rhob_r2 = r2_score(rhob['RHOB_UM'], rhob['RHOB_ML_UM']) if len(rhob) > 5 else np.nan
        else:
            rhob_r2 = np.nan

        # --- Compute DT R² ---
        if {'DT_UM', 'DT_ML_UM'}.issubset(formation_data.columns):
            dt = formation_data[['DT_UM', 'DT_ML_UM']].dropna()
            dt_r2 = r2_score(dt['DT_UM'], dt['DT_ML_UM']) if len(dt) > 5 else np.nan
        else:
            dt_r2 = np.nan

        results.append({
            "Well": well_name,
            "Formation": formation,
            "Top Depth": top_depth,
            "Base Depth": base_depth,
            "RHOB_R²": rhob_r2,
            "DT_R²": dt_r2
        })

# --- Save Results ---
results_df = pd.DataFrame(results)
results_df.to_excel(output_file, index=False)

print(f"✅ Formation-wise R² results saved to: {output_file}")
print(results_df.head(10))

⚠️ Skipping GAYA-1: Not enough formation intervals with valid depths.
⚠️ Skipping KERAMAN-1: Not enough formation intervals with valid depths.
⚠️ Skipping KINABALU EAST-1 (KINABALU-3): No matching LAS data found.
⚠️ Skipping SAMARANG-1: No matching LAS data found.
⚠️ Skipping SUMANDAK SELATAN-1: No matching LAS data found.
⚠️ Skipping SUMANDAK SELATAN-1S1: No matching LAS data found.
⚠️ Skipping SUMANDAK TEPI-1: No matching LAS data found.
⚠️ Skipping SUMANDAK UJONG-1: No matching LAS data found.
⚠️ Skipping SUMANDAK UTARA-1: No matching LAS data found.
⚠️ Skipping TAPIR-1: Not enough formation intervals with valid depths.
⚠️ Skipping TRUS MADI-1: No matching LAS data found.
✅ Formation-wise R² results saved to: Formation_R2_Scores.xlsx
          Well      Formation  Top Depth  Base Depth   RHOB_R²     DT_R²
0       ALAB-1      Stage IVE     789.40     1214.60  0.260201 -3.614982
1       ALAB-1      Stage IVD    1214.60     1405.10 -0.909932 -1.730245
2       ALAB-1         J SAND    1